In [1]:
import os

In [2]:
os.getcwd()

'd:\\mydlproject\\kidney_dlproject\\research'

In [3]:
%pwd

'd:\\mydlproject\\kidney_dlproject\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\mydlproject\\kidney_dlproject'

In [7]:
#1. Update config.yaml
# 2. Update secrets.yaml [Optional]
# 3. Update params.yaml
# 4. Update the entity
# 5. Update the configuration manager in src config
# 6. Update the components
# 7. Update the pipeline 
# 8. Update the main.py